In [86]:
import pandas as pd
import hvplot.pandas
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, mean_squared_error, r2_score
import joblib


In [87]:
df3=pd.read_csv("../Resources/cleaned_data_SD_final.csv", encoding='utf-8')
df3=df3.iloc[: , 1:]
df3.head()

,listing_url,last_scraped,host_since,zipcode,room_type,accommodates,bathrooms_text,bedrooms,beds,amenities_cat,...,0.75_revenue,revenue_cat,0.25_revenue_acc,0.5_revenue_acc,0.75_revenue_acc,revenue_cat_acc,0.25_price_bedcat,0.5_price_bedcat,0.75_price_bedcat,price_cat
0,https://www.airbnb.com/rooms/542822559960498876,2023-03-25,2015-07-16,92109.0,Entire home/apt,4,1.0,1.0,2.0,basic,...,36127.5,>75th,2894.583333,8824.5,15303.5,>75th,85.0,120.0,164.0,>75th
1,https://www.airbnb.com/rooms/49634069,2023-03-25,2021-05-06,92109.0,Entire home/apt,4,1.0,1.0,2.0,moderate,...,36127.5,>75th,2894.583333,8824.5,15303.5,50th-75th,85.0,120.0,164.0,>75th
2,https://www.airbnb.com/rooms/50448428,2023-03-25,2012-06-17,92109.0,Entire home/apt,4,1.0,1.0,1.0,basic,...,36127.5,>75th,2894.583333,8824.5,15303.5,>75th,85.0,120.0,164.0,>75th
3,https://www.airbnb.com/rooms/44697260,2023-03-25,2014-03-30,92107.0,Entire home/apt,4,1.0,1.0,2.0,basic,...,36127.5,25th-50th,2894.583333,8824.5,15303.5,25th-50th,85.0,120.0,164.0,>75th
4,https://www.airbnb.com/rooms/44697268,2023-03-25,2014-03-30,92107.0,Entire home/apt,4,1.0,1.0,2.0,basic,...,36127.5,50th-75th,2894.583333,8824.5,15303.5,25th-50th,85.0,120.0,164.0,>75th


In [88]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10738 entries, 0 to 10737
Data columns (total 49 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   listing_url                10738 non-null  object 
 1   last_scraped               10738 non-null  object 
 2   host_since                 10737 non-null  object 
 3   zipcode                    10722 non-null  float64
 4   room_type                  10738 non-null  object 
 5   accommodates               10738 non-null  int64  
 6   bathrooms_text             10738 non-null  float64
 7   bedrooms                   10738 non-null  float64
 8   beds                       10627 non-null  float64
 9   amenities_cat              10738 non-null  object 
 10  amenities_count            10738 non-null  int64  
 11  minimum_nights             10738 non-null  int64  
 12  maximum_nights             10738 non-null  int64  
 13  instant_bookable           10738 non-null  obj

In [89]:
df3["zipcode"]=df3["zipcode"].astype(object)

In [90]:
# df3=df3[df3["bedrooms_cat"]=="1.0"]
# df3
df3["bedrooms_cat"].unique()

array(['1.0', '2.0', '3.0', '4.0', '5.0', '>=6'], dtype=object)

In [91]:
df3=df3.drop(columns="amenities_count")
df3

,listing_url,last_scraped,host_since,zipcode,room_type,accommodates,bathrooms_text,bedrooms,beds,amenities_cat,...,0.75_revenue,revenue_cat,0.25_revenue_acc,0.5_revenue_acc,0.75_revenue_acc,revenue_cat_acc,0.25_price_bedcat,0.5_price_bedcat,0.75_price_bedcat,price_cat
0,https://www.airbnb.com/rooms/542822559960498876,2023-03-25,2015-07-16,92109.0,Entire home/apt,4,1.0,1.0,2.0,basic,...,36127.5,>75th,2894.583333,8824.500000,15303.5,>75th,85.0,120.0,164.0,>75th
1,https://www.airbnb.com/rooms/49634069,2023-03-25,2021-05-06,92109.0,Entire home/apt,4,1.0,1.0,2.0,moderate,...,36127.5,>75th,2894.583333,8824.500000,15303.5,50th-75th,85.0,120.0,164.0,>75th
2,https://www.airbnb.com/rooms/50448428,2023-03-25,2012-06-17,92109.0,Entire home/apt,4,1.0,1.0,1.0,basic,...,36127.5,>75th,2894.583333,8824.500000,15303.5,>75th,85.0,120.0,164.0,>75th
3,https://www.airbnb.com/rooms/44697260,2023-03-25,2014-03-30,92107.0,Entire home/apt,4,1.0,1.0,2.0,basic,...,36127.5,25th-50th,2894.583333,8824.500000,15303.5,25th-50th,85.0,120.0,164.0,>75th
4,https://www.airbnb.com/rooms/44697268,2023-03-25,2014-03-30,92107.0,Entire home/apt,4,1.0,1.0,2.0,basic,...,36127.5,50th-75th,2894.583333,8824.500000,15303.5,25th-50th,85.0,120.0,164.0,>75th
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10733,https://www.airbnb.com/rooms/53201577,2023-03-25,2016-01-25,92107.0,Entire home/apt,16,7.0,8.0,11.0,moderate,...,277648.0,50th-75th,4131.666667,8871.166667,17546.0,50th-75th,744.0,1019.0,1727.0,50th-75th
10734,https://www.airbnb.com/rooms/53193767,2023-03-25,2021-10-28,92037.0,Entire home/apt,16,7.0,10.0,13.0,basic,...,277648.0,<25th,4131.666667,8871.166667,17546.0,<25th,744.0,1019.0,1727.0,>75th
10735,https://www.airbnb.com/rooms/714482450713596663,2023-03-25,2011-09-20,92102.0,Entire home/apt,16,7.0,11.0,28.0,moderate,...,277648.0,50th-75th,4131.666667,8871.166667,17546.0,50th-75th,744.0,1019.0,1727.0,>75th
10736,https://www.airbnb.com/rooms/811514787237852543,2023-03-25,2011-09-20,92102.0,Entire home/apt,16,7.0,11.0,26.0,moderate,...,277648.0,25th-50th,4131.666667,8871.166667,17546.0,25th-50th,744.0,1019.0,1727.0,50th-75th


In [92]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10738 entries, 0 to 10737
Data columns (total 48 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   listing_url                10738 non-null  object 
 1   last_scraped               10738 non-null  object 
 2   host_since                 10737 non-null  object 
 3   zipcode                    10722 non-null  object 
 4   room_type                  10738 non-null  object 
 5   accommodates               10738 non-null  int64  
 6   bathrooms_text             10738 non-null  float64
 7   bedrooms                   10738 non-null  float64
 8   beds                       10627 non-null  float64
 9   amenities_cat              10738 non-null  object 
 10  minimum_nights             10738 non-null  int64  
 11  maximum_nights             10738 non-null  int64  
 12  instant_bookable           10738 non-null  object 
 13  availability_365           10738 non-null  int

In [93]:
# Transform the categorical columns using get_dummies
categorical_columns_2=df3.dtypes[df3.dtypes=="object"].index.tolist()
categorical_columns_2.remove("listing_url")
categorical_columns_2.remove("last_scraped")
categorical_columns_2.remove("host_since")
categorical_columns_2.remove("expected_annual_occupency")




categorical_dummies_2 = pd.get_dummies(df3[categorical_columns_2])

# Display the transformed data
categorical_dummies_2

C:\Users\nkananizadeh\Anaconda3\lib\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


,zipcode_2037.0,zipcode_22000.0,zipcode_91239.0,zipcode_91902.0,zipcode_91910.0,zipcode_91911.0,zipcode_91913.0,zipcode_91914.0,zipcode_91915.0,zipcode_91932.0,...,revenue_cat_<25th,revenue_cat_>75th,revenue_cat_acc_25th-50th,revenue_cat_acc_50th-75th,revenue_cat_acc_<25th,revenue_cat_acc_>75th,price_cat_25th-50th,price_cat_50th-75th,price_cat_<25th,price_cat_>75th
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10733,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
10734,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,1
10735,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
10736,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0


In [94]:
# the numeric columns
numeric_columns_2=df3.dtypes[df3.dtypes!="object"].index.tolist()
# have to remove the id, lat and long, etc. columns out
data_2=df3[numeric_columns_2].loc[:,~df3[numeric_columns_2].columns.isin(["listing_url","host_since","last_scraped","latitude","longitude",
                                                                    "0.25_price","0.5_price","0.75_price",
                                                                    "0.25_price_bedcat","0.5_price_bedcat","0.75_price_bedcat",
                                                                    "0.25_acc","0.5_acc","0.75_acc",
                                                                    "0.25_bathroom","0.5_bathroom","0.75_bathroom",
                                                                    "0.25_revenue","0.5_revenue","0.75_revenue",
                                                                    "0.25_revenue_acc","0.5_revenue_acc","0.75_revenue_acc",
                                                                    "outlier", "outlier_2","outlier_3",# outliers
                                                                    "bedrooms", "bathrooms_text", "years_in_business" ## these are inthe categorical columns
                                                                    ])]
data_2

,accommodates,beds,minimum_nights,maximum_nights,availability_365,price,revenue,revenue_per_accommodates
0,4,2.0,1,365,92,309.0,84357.0,21089.2500
1,4,2.0,3,31,172,306.0,59058.0,14764.5000
2,4,1.0,3,14,0,304.0,110960.0,27740.0000
3,4,2.0,1,1125,313,302.0,15704.0,3926.0000
4,4,2.0,1,1125,277,302.0,26576.0,6644.0000
...,...,...,...,...,...,...,...,...
10733,16,11.0,3,365,209,1570.0,244920.0,15307.5000
10734,16,13.0,3,365,364,2906.0,2906.0,181.6250
10735,16,28.0,3,365,270,1752.0,166440.0,10402.5000
10736,16,26.0,3,15,283,1639.0,134398.0,8399.8750


In [95]:
# combining the numeric and categorical columns
df_dummies_2=pd.concat([data_2.reset_index(drop=True),categorical_dummies_2.reset_index(drop=True)], axis=1)
df_dummies_2

,accommodates,beds,minimum_nights,maximum_nights,availability_365,price,revenue,revenue_per_accommodates,zipcode_2037.0,zipcode_22000.0,...,revenue_cat_<25th,revenue_cat_>75th,revenue_cat_acc_25th-50th,revenue_cat_acc_50th-75th,revenue_cat_acc_<25th,revenue_cat_acc_>75th,price_cat_25th-50th,price_cat_50th-75th,price_cat_<25th,price_cat_>75th
0,4,2.0,1,365,92,309.0,84357.0,21089.2500,0,0,...,0,1,0,0,0,1,0,0,0,1
1,4,2.0,3,31,172,306.0,59058.0,14764.5000,0,0,...,0,1,0,1,0,0,0,0,0,1
2,4,1.0,3,14,0,304.0,110960.0,27740.0000,0,0,...,0,1,0,0,0,1,0,0,0,1
3,4,2.0,1,1125,313,302.0,15704.0,3926.0000,0,0,...,0,0,1,0,0,0,0,0,0,1
4,4,2.0,1,1125,277,302.0,26576.0,6644.0000,0,0,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10733,16,11.0,3,365,209,1570.0,244920.0,15307.5000,0,0,...,0,0,0,1,0,0,0,1,0,0
10734,16,13.0,3,365,364,2906.0,2906.0,181.6250,0,0,...,1,0,0,0,1,0,0,0,0,1
10735,16,28.0,3,365,270,1752.0,166440.0,10402.5000,0,0,...,0,0,0,1,0,0,0,0,0,1
10736,16,26.0,3,15,283,1639.0,134398.0,8399.8750,0,0,...,0,0,1,0,0,0,0,1,0,0


# Random Forest
Target: price

In [96]:
# check the columns
for i in df_dummies_2.columns:
    print(i)

accommodates
beds
minimum_nights
maximum_nights
availability_365
price
revenue
revenue_per_accommodates
zipcode_2037.0
zipcode_22000.0
zipcode_91239.0
zipcode_91902.0
zipcode_91910.0
zipcode_91911.0
zipcode_91913.0
zipcode_91914.0
zipcode_91915.0
zipcode_91932.0
zipcode_91941.0
zipcode_91942.0
zipcode_91945.0
zipcode_91950.0
zipcode_91977.0
zipcode_92014.0
zipcode_92017.0
zipcode_92018.0
zipcode_92025.0
zipcode_92027.0
zipcode_92029.0
zipcode_92037.0
zipcode_92039.0
zipcode_92071.0
zipcode_92075.0
zipcode_92091.0
zipcode_92093.0
zipcode_92101.0
zipcode_92102.0
zipcode_92103.0
zipcode_92104.0
zipcode_92105.0
zipcode_92106.0
zipcode_92107.0
zipcode_92108.0
zipcode_92109.0
zipcode_92110.0
zipcode_92111.0
zipcode_92112.0
zipcode_92113.0
zipcode_92114.0
zipcode_92115.0
zipcode_92116.0
zipcode_92117.0
zipcode_92119.0
zipcode_92120.0
zipcode_92121.0
zipcode_92122.0
zipcode_92123.0
zipcode_92124.0
zipcode_92126.0
zipcode_92127.0
zipcode_92128.0
zipcode_92129.0
zipcode_92130.0
zipcode_92131.0
z

In [97]:
#defining features, some rows have to be removed due to nans. This steps will be taken in the next cells
X1=df_dummies_2.drop(columns=["revenue","revenue_per_accommodates","availability_365","price",
                              "price_cat_<25th","price_cat_25th-50th","price_cat_50th-75th","price_cat_>75th",
                              "revenue_cat_<25th","revenue_cat_25th-50th","revenue_cat_50th-75th","revenue_cat_>75th",
                              "revenue_cat_acc_<25th","revenue_cat_acc_25th-50th","revenue_cat_acc_50th-75th","revenue_cat_acc_>75th"
                                     ])

# check the features columns
for i in X1.columns:
    print(i)


accommodates
beds
minimum_nights
maximum_nights
zipcode_2037.0
zipcode_22000.0
zipcode_91239.0
zipcode_91902.0
zipcode_91910.0
zipcode_91911.0
zipcode_91913.0
zipcode_91914.0
zipcode_91915.0
zipcode_91932.0
zipcode_91941.0
zipcode_91942.0
zipcode_91945.0
zipcode_91950.0
zipcode_91977.0
zipcode_92014.0
zipcode_92017.0
zipcode_92018.0
zipcode_92025.0
zipcode_92027.0
zipcode_92029.0
zipcode_92037.0
zipcode_92039.0
zipcode_92071.0
zipcode_92075.0
zipcode_92091.0
zipcode_92093.0
zipcode_92101.0
zipcode_92102.0
zipcode_92103.0
zipcode_92104.0
zipcode_92105.0
zipcode_92106.0
zipcode_92107.0
zipcode_92108.0
zipcode_92109.0
zipcode_92110.0
zipcode_92111.0
zipcode_92112.0
zipcode_92113.0
zipcode_92114.0
zipcode_92115.0
zipcode_92116.0
zipcode_92117.0
zipcode_92119.0
zipcode_92120.0
zipcode_92121.0
zipcode_92122.0
zipcode_92123.0
zipcode_92124.0
zipcode_92126.0
zipcode_92127.0
zipcode_92128.0
zipcode_92129.0
zipcode_92130.0
zipcode_92131.0
zipcode_92133.0
zipcode_92134.0
zipcode_92136.0
zipcode_9

In [98]:
#dataframe for ML
rf_data=pd.concat([X1.reset_index(drop=True),pd.DataFrame(df3["price"]).reset_index(drop=True)], axis=1).dropna()
rf_data

,accommodates,beds,minimum_nights,maximum_nights,zipcode_2037.0,zipcode_22000.0,zipcode_91239.0,zipcode_91902.0,zipcode_91910.0,zipcode_91911.0,...,bathrooms_cat_2,bathrooms_cat_3,bathrooms_cat_4,bathrooms_cat_5,bathrooms_cat_>=6,years_in_business_cat_ 0,years_in_business_cat_ 1-5,years_in_business_cat_ 6-10,years_in_business_cat_>10,price
0,4,2.0,1,365,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,309.0
1,4,2.0,3,31,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,306.0
2,4,1.0,3,14,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,304.0
3,4,2.0,1,1125,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,302.0
4,4,2.0,1,1125,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,302.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10733,16,11.0,3,365,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1570.0
10734,16,13.0,3,365,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,2906.0
10735,16,28.0,3,365,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,1752.0
10736,16,26.0,3,15,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,1639.0


In [99]:
#defining features
X=rf_data.drop("price", axis=1)
X

,accommodates,beds,minimum_nights,maximum_nights,zipcode_2037.0,zipcode_22000.0,zipcode_91239.0,zipcode_91902.0,zipcode_91910.0,zipcode_91911.0,...,bathrooms_cat_1,bathrooms_cat_2,bathrooms_cat_3,bathrooms_cat_4,bathrooms_cat_5,bathrooms_cat_>=6,years_in_business_cat_ 0,years_in_business_cat_ 1-5,years_in_business_cat_ 6-10,years_in_business_cat_>10
0,4,2.0,1,365,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
1,4,2.0,3,31,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
2,4,1.0,3,14,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
3,4,2.0,1,1125,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
4,4,2.0,1,1125,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10733,16,11.0,3,365,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
10734,16,13.0,3,365,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
10735,16,28.0,3,365,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
10736,16,26.0,3,15,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1


In [100]:
#defining target

y=rf_data["price"].ravel()
y

array([ 309.,  306.,  304., ..., 1752., 1639., 1595.])

In [101]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Random Forest doesn't care about regression 

In [102]:
# #Scaling the data
# X_train_scale=StandardScaler().fit_transform(X_train)
# X_test_scale=StandardScaler().fit_transform(X_test)

In [103]:
pd.DataFrame(y_test).value_counts()

150.0     33
250.0     30
125.0     30
120.0     28
100.0     27
          ..
535.0      1
536.0      1
539.0      1
546.0      1
3528.0     1
Length: 656, dtype: int64

In [104]:
# Fitting the random forest model
regressor = RandomForestRegressor(n_estimators = 500, random_state = 0)
regressor.fit(X_train, y_train)
#predict the model
prediction = regressor.predict(X_test)

In [105]:
# print the metrics
print("MSE:",mean_squared_error(y_test, prediction))

print("R2:", r2_score(y_test, prediction))

MSE: 18368.537262674297
R2: 0.7291980560531666


# Hyperparameter tunning

In [21]:
param_grid={'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

In [22]:
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = param_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [23]:
rf_random.best_params_

{'n_estimators': 1600,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 30,
 'bootstrap': True}

In [106]:
# Fitting the xgb model
regressor = RandomForestRegressor(n_estimators = 1600, random_state = 0,min_samples_split=2, min_samples_leaf=1,
                                  max_features='sqrt', max_depth=30, bootstrap=True)
regressor.fit(X_train, y_train)
#predict the model
prediction = regressor.predict(X_test)

In [107]:
# print the metrics
print("rMSE:",mean_squared_error(y_test, prediction, squared=False))

print("R2:", r2_score(y_test, prediction))

rMSE: 131.88666072619816
R2: 0.7435640266952945


In [108]:
actual=pd.DataFrame(y_test).rename({0:"Actual: price"}, axis=1)
predicted=pd.DataFrame(prediction).rename({0:"Predicted: price"}, axis=1)
fit_df_price=pd.concat([actual,predicted], axis=1)
fit_df_price

,Actual: price,Predicted: price
0,698.0,641.588430
1,60.0,62.927517
2,90.0,91.226288
3,41.0,62.808725
4,264.0,290.345640
...,...,...
2652,176.0,176.054616
2653,91.0,148.101425
2654,228.0,327.034237
2655,127.0,156.079678


In [109]:
fig1=fit_df_price.hvplot.scatter(x="Actual: price", y="Predicted: price")
fig2=pd.DataFrame(range(0,4000)).hvplot.line(x=0, y=0, c="red")
fig1 * fig2

:Overlay
   .Scatter.I :Scatter   [Actual: price]   (Predicted: price)
   .Curve.I   :Curve   [index]   (0)

# Save and load the model to test with the test data

In [110]:
# save the model
joblib.dump(regressor, "./random_forest.joblib")

['./random_forest.joblib']

In [111]:
# load, no need to initialize the loaded_rf the model
model = joblib.load("./random_forest.joblib")

In [112]:
X_test

,accommodates,beds,minimum_nights,maximum_nights,zipcode_2037.0,zipcode_22000.0,zipcode_91239.0,zipcode_91902.0,zipcode_91910.0,zipcode_91911.0,...,bathrooms_cat_1,bathrooms_cat_2,bathrooms_cat_3,bathrooms_cat_4,bathrooms_cat_5,bathrooms_cat_>=6,years_in_business_cat_ 0,years_in_business_cat_ 1-5,years_in_business_cat_ 6-10,years_in_business_cat_>10
9272,10,6.0,4,365,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
4215,1,1.0,1,1125,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
4691,2,1.0,1,100,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
3303,2,1.0,1,1125,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
5549,5,3.0,3,28,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441,4,2.0,1,365,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
2628,2,1.0,2,30,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
6484,4,2.0,7,30,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
845,4,2.0,1,28,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0


# 
Input the test data

In [113]:
# the test data
df_test=pd.DataFrame([{'accommodates': 6,
'beds':2,
'minimum_nights':2,
'maximum_nights':30,
'zipcode':'92102.0',
'room_type':'Entire home/apt',
'amenities_cat':'moderate',
'instant_bookable':'t',
'bedrooms_cat':'2.0',
'bathrooms_cat':'2',
'years_in_business_cat':' 1-5'}])
df_test

,accommodates,beds,minimum_nights,maximum_nights,zipcode,room_type,amenities_cat,instant_bookable,bedrooms_cat,bathrooms_cat,years_in_business_cat
0,6,2,2,30,92102.0,Entire home/apt,moderate,t,2.0,2,1-5


In [114]:
# Transform the categorical columns using get_dummies
categorical_columns_test=df_test.dtypes[df_test.dtypes=="object"].index.tolist()


categorical_columns_test = pd.get_dummies(df_test[categorical_columns_test])

# Display the transformed data
categorical_columns_test

,zipcode_92102.0,room_type_Entire home/apt,amenities_cat_moderate,instant_bookable_t,bedrooms_cat_2.0,bathrooms_cat_2,years_in_business_cat_ 1-5
0,1,1,1,1,1,1,1


In [115]:
# the numeric columns
numeric_columns_test=df_test.dtypes[df_test.dtypes!="object"].index.tolist()
# have to remove the id, lat and long, etc. columns out
data_test=df_test[numeric_columns_test]
data_test


,accommodates,beds,minimum_nights,maximum_nights
0,6,2,2,30


In [116]:
test_data=pd.concat([data_test,categorical_columns_test], axis=1)
test_data

,accommodates,beds,minimum_nights,maximum_nights,zipcode_92102.0,room_type_Entire home/apt,amenities_cat_moderate,instant_bookable_t,bedrooms_cat_2.0,bathrooms_cat_2,years_in_business_cat_ 1-5
0,6,2,2,30,1,1,1,1,1,1,1


In [117]:
#making dummy table with all 0s to fill the test dataset
X10=X1.iloc[:1]*0
X10

,accommodates,beds,minimum_nights,maximum_nights,zipcode_2037.0,zipcode_22000.0,zipcode_91239.0,zipcode_91902.0,zipcode_91910.0,zipcode_91911.0,...,bathrooms_cat_1,bathrooms_cat_2,bathrooms_cat_3,bathrooms_cat_4,bathrooms_cat_5,bathrooms_cat_>=6,years_in_business_cat_ 0,years_in_business_cat_ 1-5,years_in_business_cat_ 6-10,years_in_business_cat_>10
0,0,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [118]:
temp = pd.concat([X10, test_data]).fillna(0)
X_Test=temp.iloc[:1]+temp.iloc[1:2]
X_Test

,accommodates,beds,minimum_nights,maximum_nights,zipcode_2037.0,zipcode_22000.0,zipcode_91239.0,zipcode_91902.0,zipcode_91910.0,zipcode_91911.0,...,bathrooms_cat_1,bathrooms_cat_2,bathrooms_cat_3,bathrooms_cat_4,bathrooms_cat_5,bathrooms_cat_>=6,years_in_business_cat_ 0,years_in_business_cat_ 1-5,years_in_business_cat_ 6-10,years_in_business_cat_>10
0,6,2.0,2,30,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0


In [119]:
predictions=model.predict(X_Test)

In [120]:
predicted_test=pd.DataFrame(predictions).rename({0:"Predicted: price"}, axis=1)

predicted_test

,Predicted: price
0,205.809521
